## Quickstatements zum Ändern bestehender Daten

Mithilfe dieses Skripts können Daten in Wikidata zu bestehenden Datenobjeken ergänzt werden. In diesem Beispiel werden Daten aus dem iDAI.gazetteer ergänzt.

mit https://quickstatements.toolforge.org

Das Skript kann nach belieben angepasst werden.

Version 1.0, 25.07.2025

In [ ]:
import pandas as pd
import requests

In [ ]:
df_data = pd.read_excel('data.xlsx', sheet_name='wikidata')

print(df_data.head())

In [ ]:
def get_gaz_id(uri):
    parts = uri.split("/")
    gazid = parts[-1]
    return gazid

In [ ]:
names_list = []
types = []
wikidata_ids = []

for place in df_data['GazetteerID']:
    gaz_id = get_gaz_id(place)
    print(gaz_id)
    # Assuming the gazetteer URL follows a specific pattern
    gaz_url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(gaz_url)
    if response.status_code == 200:
        data = response.json()
        names = data.get('names')
        names = names[0]
        print(names)
        if names['language'] == 'eng':
            name = names['title']
            print(name)
            names_list.append(name)
            if 'Ceramic' in name:
                types.append('pottery style')
            else:
                types.append('CHECK MANUALLY!')
        else:
            names_list.append('TRANSLATE MANUALLY!')
            types.append('CHECK MANUALLY!')

print(names)
print(types)

In [ ]:
wikidata_ids = df_data['Wikidata_URL'].tolist()

print(len(names_list))
print(len(types))
print(len(wikidata_ids))

#merge the lists into a DataFrame
df = pd.DataFrame({
    'name': names_list,
    'type': types,
    'wikidata_id': wikidata_ids
})

#Create Quickstatements

quickstatements = []

for index, row in df.iterrows():
    wiki_id = row['wikidata_id']
    wiki_id = wiki_id.split("/")[-1]
    quickstatements.append(f'{wiki_id}|Len|"{row["name"]}"')
    quickstatements.append(f'{wiki_id}|Den|"{row["type"]}"')
    quickstatements.append("")
    
print(quickstatements)

#save as text file
with open('quickstatements.txt', 'w') as f:
    for statement in quickstatements:
        f.write(statement + '\n')

Entwickelt von Lukas Lammers im Projekt FAIR.rdm, teil des DFG-geförderten SPP 2143 "Entangled Africa".